In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("practice").getOrCreate()

2021-09-23 09:10:06,749 WARN util.Utils: Your hostname, tb-LinuxBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2021-09-23 09:10:06,751 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
2021-09-23 09:10:09,392 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-09-23 09:10:14,708 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
columns = ["first_name","last_name","department","state","salary","age","bonus"]
data = [
        ("Jonathan","snow","Developer","NC",90000,35,2000),\
        ("Tony","stark","Developer","NY",120000,45,3000),\
        ("Arya","stark","Developer","NC",95000,18,4000),\
        ("Bruce","banner","Sales","CA",100000,58,1000),\
        ("Natasha","romanov","Sales","CA",80000,40,5000),\
        ("Steve","rogers","Sales","WA",70000,28,8000),\
        ("Thanos","","Finance","NC",60000,75,3000),\
        ("Sansa","stark","Developer","WA",90000,23,9000),\
        ("Gandalf","","Finance","NY",150000,80,2000),\
        ("Cercei","lannister","Finance","NC",85000,46,2500),\
        ("Thor","","Sales","WA",75000,50,5000),\
        ("Lionel","messi","Developer","NY",130000,34,6000),\
        ]

In [4]:
df = spark.createDataFrame(data,columns)

In [5]:
df.show()

+----------+---------+----------+-----+------+---+-----+
|first_name|last_name|department|state|salary|age|bonus|
+----------+---------+----------+-----+------+---+-----+
|  Jonathan|     snow| Developer|   NC| 90000| 35| 2000|
|      Tony|    stark| Developer|   NY|120000| 45| 3000|
|      Arya|    stark| Developer|   NC| 95000| 18| 4000|
|     Bruce|   banner|     Sales|   CA|100000| 58| 1000|
|   Natasha|  romanov|     Sales|   CA| 80000| 40| 5000|
|     Steve|   rogers|     Sales|   WA| 70000| 28| 8000|
|    Thanos|         |   Finance|   NC| 60000| 75| 3000|
|     Sansa|    stark| Developer|   WA| 90000| 23| 9000|
|   Gandalf|         |   Finance|   NY|150000| 80| 2000|
|    Cercei|lannister|   Finance|   NC| 85000| 46| 2500|
|      Thor|         |     Sales|   WA| 75000| 50| 5000|
|    Lionel|    messi| Developer|   NY|130000| 34| 6000|
+----------+---------+----------+-----+------+---+-----+



In [6]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)



In [7]:
#print last names of Finance people

In [8]:
df.select(df.last_name,df.department).filter(df.department == "Finance").show()

+---------+----------+
|last_name|department|
+---------+----------+
|         |   Finance|
|         |   Finance|
|lannister|   Finance|
+---------+----------+



In [9]:
from pyspark.sql.functions import col, concat, concat_ws, upper, udf

In [10]:
df = df.withColumn("total_salary",(col("bonus") + col("salary")))

In [11]:
df.show()

+----------+---------+----------+-----+------+---+-----+------------+
|first_name|last_name|department|state|salary|age|bonus|total_salary|
+----------+---------+----------+-----+------+---+-----+------------+
|  Jonathan|     snow| Developer|   NC| 90000| 35| 2000|       92000|
|      Tony|    stark| Developer|   NY|120000| 45| 3000|      123000|
|      Arya|    stark| Developer|   NC| 95000| 18| 4000|       99000|
|     Bruce|   banner|     Sales|   CA|100000| 58| 1000|      101000|
|   Natasha|  romanov|     Sales|   CA| 80000| 40| 5000|       85000|
|     Steve|   rogers|     Sales|   WA| 70000| 28| 8000|       78000|
|    Thanos|         |   Finance|   NC| 60000| 75| 3000|       63000|
|     Sansa|    stark| Developer|   WA| 90000| 23| 9000|       99000|
|   Gandalf|         |   Finance|   NY|150000| 80| 2000|      152000|
|    Cercei|lannister|   Finance|   NC| 85000| 46| 2500|       87500|
|      Thor|         |     Sales|   WA| 75000| 50| 5000|       80000|
|    Lionel|    mess

In [12]:
# add a column called full_name with full name data

In [13]:
df = df.withColumn("full_name",concat_ws(" ",df.first_name,df.last_name))\
                            .select("first_name","last_name","full_name",\
                            "department","state","salary","age","bonus","total_salary")

In [14]:
df.show()

+----------+---------+----------------+----------+-----+------+---+-----+------------+
|first_name|last_name|       full_name|department|state|salary|age|bonus|total_salary|
+----------+---------+----------------+----------+-----+------+---+-----+------------+
|  Jonathan|     snow|   Jonathan snow| Developer|   NC| 90000| 35| 2000|       92000|
|      Tony|    stark|      Tony stark| Developer|   NY|120000| 45| 3000|      123000|
|      Arya|    stark|      Arya stark| Developer|   NC| 95000| 18| 4000|       99000|
|     Bruce|   banner|    Bruce banner|     Sales|   CA|100000| 58| 1000|      101000|
|   Natasha|  romanov| Natasha romanov|     Sales|   CA| 80000| 40| 5000|       85000|
|     Steve|   rogers|    Steve rogers|     Sales|   WA| 70000| 28| 8000|       78000|
|    Thanos|         |         Thanos |   Finance|   NC| 60000| 75| 3000|       63000|
|     Sansa|    stark|     Sansa stark| Developer|   WA| 90000| 23| 9000|       99000|
|   Gandalf|         |        Gandalf |   F

In [15]:
df.withColumn("last_name",upper(df.last_name)).show()

+----------+---------+----------------+----------+-----+------+---+-----+------------+
|first_name|last_name|       full_name|department|state|salary|age|bonus|total_salary|
+----------+---------+----------------+----------+-----+------+---+-----+------------+
|  Jonathan|     SNOW|   Jonathan snow| Developer|   NC| 90000| 35| 2000|       92000|
|      Tony|    STARK|      Tony stark| Developer|   NY|120000| 45| 3000|      123000|
|      Arya|    STARK|      Arya stark| Developer|   NC| 95000| 18| 4000|       99000|
|     Bruce|   BANNER|    Bruce banner|     Sales|   CA|100000| 58| 1000|      101000|
|   Natasha|  ROMANOV| Natasha romanov|     Sales|   CA| 80000| 40| 5000|       85000|
|     Steve|   ROGERS|    Steve rogers|     Sales|   WA| 70000| 28| 8000|       78000|
|    Thanos|         |         Thanos |   Finance|   NC| 60000| 75| 3000|       63000|
|     Sansa|    STARK|     Sansa stark| Developer|   WA| 90000| 23| 9000|       99000|
|   Gandalf|         |        Gandalf |   F

In [16]:
# Make first letter of last name in upper case

In [17]:
def upper_case(word):
    if word != "":
        word = word[0].upper() + word[1:]
    return word

In [18]:
upper_case_udf = udf(lambda word: upper_case(word))

In [19]:
spark.udf.register("upper_case_udf",upper_case_udf)

<function __main__.<lambda>(word)>

In [20]:
df.withColumn("last_name",upper_case_udf(df.last_name)).show()

+----------+---------+----------------+----------+-----+------+---+-----+------------+
|first_name|last_name|       full_name|department|state|salary|age|bonus|total_salary|
+----------+---------+----------------+----------+-----+------+---+-----+------------+
|  Jonathan|     Snow|   Jonathan snow| Developer|   NC| 90000| 35| 2000|       92000|
|      Tony|    Stark|      Tony stark| Developer|   NY|120000| 45| 3000|      123000|
|      Arya|    Stark|      Arya stark| Developer|   NC| 95000| 18| 4000|       99000|
|     Bruce|   Banner|    Bruce banner|     Sales|   CA|100000| 58| 1000|      101000|
|   Natasha|  Romanov| Natasha romanov|     Sales|   CA| 80000| 40| 5000|       85000|
|     Steve|   Rogers|    Steve rogers|     Sales|   WA| 70000| 28| 8000|       78000|
|    Thanos|         |         Thanos |   Finance|   NC| 60000| 75| 3000|       63000|
|     Sansa|    Stark|     Sansa stark| Developer|   WA| 90000| 23| 9000|       99000|
|   Gandalf|         |        Gandalf |   F

In [21]:
df.show()

+----------+---------+----------------+----------+-----+------+---+-----+------------+
|first_name|last_name|       full_name|department|state|salary|age|bonus|total_salary|
+----------+---------+----------------+----------+-----+------+---+-----+------------+
|  Jonathan|     snow|   Jonathan snow| Developer|   NC| 90000| 35| 2000|       92000|
|      Tony|    stark|      Tony stark| Developer|   NY|120000| 45| 3000|      123000|
|      Arya|    stark|      Arya stark| Developer|   NC| 95000| 18| 4000|       99000|
|     Bruce|   banner|    Bruce banner|     Sales|   CA|100000| 58| 1000|      101000|
|   Natasha|  romanov| Natasha romanov|     Sales|   CA| 80000| 40| 5000|       85000|
|     Steve|   rogers|    Steve rogers|     Sales|   WA| 70000| 28| 8000|       78000|
|    Thanos|         |         Thanos |   Finance|   NC| 60000| 75| 3000|       63000|
|     Sansa|    stark|     Sansa stark| Developer|   WA| 90000| 23| 9000|       99000|
|   Gandalf|         |        Gandalf |   F

In [22]:
from pyspark.sql.functions import *

In [23]:
df.withColumn("date",current_date()).show()

+----------+---------+----------------+----------+-----+------+---+-----+------------+----------+
|first_name|last_name|       full_name|department|state|salary|age|bonus|total_salary|      date|
+----------+---------+----------------+----------+-----+------+---+-----+------------+----------+
|  Jonathan|     snow|   Jonathan snow| Developer|   NC| 90000| 35| 2000|       92000|2021-09-23|
|      Tony|    stark|      Tony stark| Developer|   NY|120000| 45| 3000|      123000|2021-09-23|
|      Arya|    stark|      Arya stark| Developer|   NC| 95000| 18| 4000|       99000|2021-09-23|
|     Bruce|   banner|    Bruce banner|     Sales|   CA|100000| 58| 1000|      101000|2021-09-23|
|   Natasha|  romanov| Natasha romanov|     Sales|   CA| 80000| 40| 5000|       85000|2021-09-23|
|     Steve|   rogers|    Steve rogers|     Sales|   WA| 70000| 28| 8000|       78000|2021-09-23|
|    Thanos|         |         Thanos |   Finance|   NC| 60000| 75| 3000|       63000|2021-09-23|
|     Sansa|    star

In [25]:
df.withColumn("date",current_timestamp()).show(truncate=False)

+----------+---------+----------------+----------+-----+------+---+-----+------------+-----------------------+
|first_name|last_name|full_name       |department|state|salary|age|bonus|total_salary|date                   |
+----------+---------+----------------+----------+-----+------+---+-----+------------+-----------------------+
|Jonathan  |snow     |Jonathan snow   |Developer |NC   |90000 |35 |2000 |92000       |2021-09-23 09:13:52.966|
|Tony      |stark    |Tony stark      |Developer |NY   |120000|45 |3000 |123000      |2021-09-23 09:13:52.966|
|Arya      |stark    |Arya stark      |Developer |NC   |95000 |18 |4000 |99000       |2021-09-23 09:13:52.966|
|Bruce     |banner   |Bruce banner    |Sales     |CA   |100000|58 |1000 |101000      |2021-09-23 09:13:52.966|
|Natasha   |romanov  |Natasha romanov |Sales     |CA   |80000 |40 |5000 |85000       |2021-09-23 09:13:52.966|
|Steve     |rogers   |Steve rogers    |Sales     |WA   |70000 |28 |8000 |78000       |2021-09-23 09:13:52.966|
|

In [26]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- full_name: string (nullable = false)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)
 |-- total_salary: long (nullable = true)



In [27]:
# make bonus data type from ling to float

In [28]:
from pyspark.sql.types import FloatType

In [30]:
df = df.withColumn("bonus",df.bonus.cast(FloatType()))

In [31]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- full_name: string (nullable = false)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: float (nullable = true)
 |-- total_salary: long (nullable = true)



In [32]:
df.show()

+----------+---------+----------------+----------+-----+------+---+------+------------+
|first_name|last_name|       full_name|department|state|salary|age| bonus|total_salary|
+----------+---------+----------------+----------+-----+------+---+------+------------+
|  Jonathan|     snow|   Jonathan snow| Developer|   NC| 90000| 35|2000.0|       92000|
|      Tony|    stark|      Tony stark| Developer|   NY|120000| 45|3000.0|      123000|
|      Arya|    stark|      Arya stark| Developer|   NC| 95000| 18|4000.0|       99000|
|     Bruce|   banner|    Bruce banner|     Sales|   CA|100000| 58|1000.0|      101000|
|   Natasha|  romanov| Natasha romanov|     Sales|   CA| 80000| 40|5000.0|       85000|
|     Steve|   rogers|    Steve rogers|     Sales|   WA| 70000| 28|8000.0|       78000|
|    Thanos|         |         Thanos |   Finance|   NC| 60000| 75|3000.0|       63000|
|     Sansa|    stark|     Sansa stark| Developer|   WA| 90000| 23|9000.0|       99000|
|   Gandalf|         |        Ga